In [4]:
import pandas as pd
import numpy as np
from typing import List
import uvicorn
import requests

In [9]:
def calculate_ee(
    envelope_kpi: float,
    envelope_min: float,
    envelope_max: float,
    window_kpi: float,
    window_min: float,
    window_max: float,
    heating_system_kpi: float,
    heating_system_min: float,
    heating_system_max: float,
    cooling_system_kpi: float,
    cooling_system_min: float,
    cooling_system_max: float,
    profile: str
) -> dict:

    def normalize_low(kpi_value, min_value, max_value):
        score = (max_value - kpi_value) / (max_value - min_value) * 100
        return max(0, min(100, score))

    ee_normalized = {}

    ee_normalized["envelope"] = envelope_normalized = normalize_low(envelope_kpi, envelope_min, envelope_max)
    ee_normalized["window"] = window_normalized = normalize_low(window_kpi, window_min, window_max)
    ee_normalized["heating_system"] = heating_system_normalized = normalize_low(heating_system_kpi, heating_system_min, heating_system_max)
    ee_normalized["cooling_system"] = cooling_system_normalized = normalize_low(cooling_system_kpi, cooling_system_min, cooling_system_max)

    no_pillar = 4
    score_total = 15

    if profile == "Environment-Oriented":
        pillar_score = 3
    elif profile == "Comfort-Oriented":
        pillar_score = 2
    elif profile == "Financally-Oriented":
        pillar_score = 2
    else:
        print("Invalid profile")

    pillar_weight = pillar_score/score_total
    ee_kpi_weight = pillar_weight * (1 / no_pillar)

    return ee_kpi_weight, envelope_normalized, window_normalized, heating_system_normalized, cooling_system_normalized

In [10]:
def calculate_rei(
    st_coverage_kpi: float,
    st_coverage_min: float,
    st_coverage_max: float,
    onsite_res_kpi: float,
    onsite_res_min: float,
    onsite_res_max: float,
    net_energy_export_kpi: float,
    net_energy_export_min: float,
    net_energy_export_max: float,
    profile: str
) -> dict:

    def normalize_high(kpi_value, min_value, max_value):
        score = (kpi_value - min_value) / (max_value - min_value) * 100
        return max(0, min(100, score))

    rei_normalized = {}

    rei_normalized["st_coverage"] = st_coverage_normalized = normalize_high(st_coverage_kpi, st_coverage_min, st_coverage_max)
    rei_normalized["onsite_res"] = onsite_res_normalized = normalize_high(onsite_res_kpi, onsite_res_min, onsite_res_max)
    rei_normalized["net_energy_export"] = net_energy_normalized = normalize_high(net_energy_export_kpi, net_energy_export_min, net_energy_export_max)

    no_pillar = 3
    score_total = 15

    if profile == "Environment-Oriented":
        pillar_score = 2
    elif profile == "Comfort-Oriented":
        pillar_score = 5
    elif profile == "Financally-Oriented":
        pillar_score = 3
    else:
        print("Invalid profile")

    pillar_weight = pillar_score/score_total
    rei_kpi_weight = pillar_weight * (1 / no_pillar)

    return rei_kpi_weight, st_coverage_normalized, onsite_res_normalized, net_energy_normalized

In [11]:
def calculate_sei(
    embodied_carbon_kpi: float,
    embodied_carbon_min: float,
    embodied_carbon_max: float,
    gwp_kpi: float,
    gwp_min: float,
    gwp_max: float,
    profile: str
) -> dict:

    def normalize_low(kpi_value, min_value, max_value):
        score = (max_value - kpi_value) / (max_value - min_value) * 100
        return max(0, min(100, score))

    sei_normalized = {}

    sei_normalized["embodied_carbon"] = embodied_carbon_normalized = normalize_low(embodied_carbon_kpi, embodied_carbon_min, embodied_carbon_max)
    sei_normalized["gwp"] = gwp_normalized = normalize_low(gwp_kpi, gwp_min, gwp_max)

    no_pillar = 2
    score_total = 15

    if profile == "Environment-Oriented":
        pillar_score = 1
    elif profile == "Comfort-Oriented":
        pillar_score = 4
    elif profile == "Financally-Oriented":
        pillar_score = 5
    else:
        print("Invalid profile")

    pillar_weight = pillar_score/score_total
    sei_kpi_weight = pillar_weight * (1 / no_pillar)

    return sei_kpi_weight, embodied_carbon_normalized, gwp_normalized

In [20]:
def calculate_uc(
    thermal_comfort_air_temp_kpi: float,
    thermal_comfort_air_temp_min: float,
    thermal_comfort_air_temp_max: float,
    thermal_comfort_humidity_kpi: float,
    thermal_comfort_humidity_min: float,
    thermal_comfort_humidity_max: float,
    profile: str
) -> dict:

    def normalize_high(kpi_value, min_value, max_value):
        score = (kpi_value - min_value) / (max_value - min_value) * 100
        return max(0, min(100, score))

    uc_normalized = {}

    uc_normalized["thermal_comfort_air_temp"] = thermal_comfort_air_temp_normalized = normalize_high(thermal_comfort_air_temp_kpi, thermal_comfort_air_temp_min, thermal_comfort_air_temp_max)
    uc_normalized["thermal_comfort_humidity"] = thermal_comfort_humidity_normalized = normalize_high(thermal_comfort_humidity_kpi, thermal_comfort_humidity_min, thermal_comfort_humidity_max)
    
    no_pillar = 2
    score_total = 15

    if profile == "Environment-Oriented":
        pillar_score = 4
    elif profile == "Comfort-Oriented":
        pillar_score = 1
    elif profile == "Financally-Oriented":
        pillar_score = 4
    else:
        print("Invalid profile")

    pillar_weight = pillar_score/score_total
    uc_kpi_weight = pillar_weight * (1 / no_pillar)

    return uc_kpi_weight, thermal_comfort_air_temp_normalized, thermal_comfort_humidity_normalized

In [21]:
def calculate_fv(
    ii_kpi: float,
    ii_min: float,
    ii_max: float,
    aoc_kpi: float,
    aoc_min: float,
    aoc_max: float,
    irr_kpi: float,
    irr_min: float,
    irr_max: float,
    npv_kpi: float,
    npv_min: float,
    npv_max: float,
    profile: str
) -> dict:

    def normalize_high(kpi_value, min_value, max_value):
        score = (kpi_value - min_value) / (max_value - min_value) * 100
        return max(0, min(100, score))

    def normalize_low(kpi_value, min_value, max_value):
        score = (max_value - kpi_value) / (max_value - min_value) * 100
        return max(0, min(100, score))

    fv_normalized = {}

    fv_normalized["ii"] = ii_normalized = normalize_low(ii_kpi, ii_min, ii_max)
    fv_normalized["aoc"] = aoc_normalized = normalize_low(aoc_kpi, aoc_min, aoc_max)
    fv_normalized["irr"] = irr_normalized = normalize_high(irr_kpi, irr_min, irr_max)
    fv_normalized["npv"] = npv_normalized = normalize_high(npv_kpi, npv_min, npv_max)

    no_pillar = 4
    score_total = 15

    if profile == "Environment-Oriented":
        pillar_score = 5
    elif profile == "Comfort-Oriented":
        pillar_score = 3
    elif profile == "Financally-Oriented":
        pillar_score = 1
    else:
        print("Invalid profile")

    pillar_weight = pillar_score/score_total
    fv_kpi_weight = pillar_weight * (1 / no_pillar)

    return fv_kpi_weight, ii_normalized, aoc_normalized, irr_normalized, npv_normalized

In [22]:
#test for ee
envelope_kpi=5
envelope_min=1
envelope_max=10
window_kpi=5
window_min=1
window_max=10
heating_system_kpi=5
heating_system_min=1
heating_system_max=10
cooling_system_kpi=5
cooling_system_min=1
cooling_system_max=10
profile="Environment-Oriented"

calculate_ee(
    envelope_kpi=envelope_kpi,
    envelope_min=envelope_min,
    envelope_max=envelope_max,
    window_kpi=window_kpi,
    window_min=window_min,
    window_max=window_max,
    heating_system_kpi=heating_system_kpi,
    heating_system_min=heating_system_min,
    heating_system_max=heating_system_max,
    cooling_system_kpi=cooling_system_kpi,
    cooling_system_min=cooling_system_min,
    cooling_system_max=cooling_system_max,
    profile=profile
)

(0.05,
 55.55555555555556,
 55.55555555555556,
 55.55555555555556,
 55.55555555555556)

In [23]:
#test for rei
st_coverage_kpi = 5
st_coverage_min = 1
st_coverage_max = 10
onsite_res_kpi = 5
onsite_res_min = 1
onsite_res_max = 10
net_energy_export_kpi = 5
net_energy_export_min = 1
net_energy_export_max = 10
profile = "Environment-Oriented"

calculate_rei(
    st_coverage_kpi=st_coverage_kpi,
    st_coverage_min=st_coverage_min,
    st_coverage_max=st_coverage_max,
    onsite_res_kpi=onsite_res_kpi,
    onsite_res_min=onsite_res_min,
    onsite_res_max=onsite_res_max,
    net_energy_export_kpi=net_energy_export_kpi,
    net_energy_export_min=net_energy_export_min,
    net_energy_export_max=net_energy_export_max,
    profile=profile
)

(0.04444444444444444, 44.44444444444444, 44.44444444444444, 44.44444444444444)

In [24]:
#test for sei
embodied_carbon_kpi = 5
embodied_carbon_min = 1
embodied_carbon_max = 10
gwp_kpi = 5
gwp_min = 1
gwp_max = 10
profile = "Environment-Oriented"

calculate_sei(
    embodied_carbon_kpi=embodied_carbon_kpi,
    embodied_carbon_min=embodied_carbon_min,
    embodied_carbon_max=embodied_carbon_max,
    gwp_kpi=gwp_kpi,
    gwp_min=gwp_min,
    gwp_max=gwp_max,
    profile=profile
)

(0.03333333333333333, 55.55555555555556, 55.55555555555556)

In [25]:
#test for uc
thermal_comfort_air_temp_kpi = 5
thermal_comfort_air_temp_min = 1
thermal_comfort_air_temp_max = 10
thermal_comfort_humidity_kpi = 5
thermal_comfort_humidity_min = 1
thermal_comfort_humidity_max = 10
net_energy_export_kpi = 5
net_energy_export_min = 1
net_energy_export_max = 10
profile = "Environment-Oriented"

calculate_uc(
    thermal_comfort_air_temp_kpi=thermal_comfort_air_temp_kpi,
    thermal_comfort_air_temp_min=thermal_comfort_air_temp_min,
    thermal_comfort_air_temp_max=thermal_comfort_air_temp_max,
    thermal_comfort_humidity_kpi=thermal_comfort_humidity_kpi,
    thermal_comfort_humidity_min=thermal_comfort_humidity_min,
    thermal_comfort_humidity_max=thermal_comfort_humidity_max,
    profile=profile
)

(0.13333333333333333, 44.44444444444444, 44.44444444444444)

In [26]:
#test for fv
ii_kpi = 5
ii_min = 1
ii_max = 10
aoc_kpi = 5
aoc_min = 1
aoc_max = 10
irr_kpi = 5
irr_min = 1
irr_max = 10
npv_kpi = 5
npv_min = 1
npv_max = 10
profile = "Environment-Oriented"

calculate_fv(
    ii_kpi=ii_kpi,
    ii_min=ii_min,
    ii_max=ii_max,
    aoc_kpi=aoc_kpi,
    aoc_min=aoc_min,
    aoc_max=aoc_max,
    irr_kpi=irr_kpi,
    irr_min=irr_min,
    irr_max=irr_max,
    npv_kpi=npv_kpi,
    npv_min=npv_min,
    npv_max=npv_max,
    profile=profile
)

(0.08333333333333333,
 55.55555555555556,
 55.55555555555556,
 44.44444444444444,
 44.44444444444444)